## <span style="color:#ff5f27">📝 Imports </span>

In [1]:
import torch
from peft import AutoPeftModelForCausalLM
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
import transformers
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferWindowMemory
from FlagEmbedding import FlagReranker

from functions.prompt_engineering import get_context_and_source
import warnings
warnings.filterwarnings('ignore')

/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/srv/hops/anaconda/envs/theenv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
2024-02-02 15:21:32.101701: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-02 15:21:32.632089: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic 

## <span style="color:#ff5f27;"> 🔮 Connecting to Hopsworks Feature Store </span>

In [2]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()
mr = project.get_model_registry()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://snurran.hops.works/p/1143
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.


In [3]:
# Retrieve the 'documents' feature view
feature_view = fs.get_feature_view(
    name="documents", 
    version=1,
)   

# Initialize serving
feature_view.init_serving(1)

In [4]:
# Get the Mistral model from Model Registry
mistral_model = mr.get_model(
    name="mistral_model",
    version=1,
)

# Download the Mistral model files to a local directory
saved_model_dir = mistral_model.download()

2024-02-02 15:22:01,385 main ERROR Cannot access RandomAccessFile java.io.IOException: Could not create directory /srv/hops/hadoop-3.2.0.12-EE-SNAPSHOT/logs java.io.IOException: Could not create directory /srv/hops/hadoop-3.2.0.12-EE-SNAPSHOT/logs
	at org.apache.logging.log4j.core.util.FileUtils.mkdir(FileUtils.java:128)
	at org.apache.logging.log4j.core.util.FileUtils.makeParentDirs(FileUtils.java:141)
	at org.apache.logging.log4j.core.appender.rolling.RollingRandomAccessFileManager$RollingRandomAccessFileManagerFactory.createManager(RollingRandomAccessFileManager.java:231)
	at org.apache.logging.log4j.core.appender.rolling.RollingRandomAccessFileManager$RollingRandomAccessFileManagerFactory.createManager(RollingRandomAccessFileManager.java:204)
	at org.apache.logging.log4j.core.appender.AbstractManager.getManager(AbstractManager.java:144)
	at org.apache.logging.log4j.core.appender.OutputStreamManager.getManager(OutputStreamManager.java:100)
	at org.apache.logging.log4j.core.appender.

SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-SNAPSHOT/share/hadoop/common/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/srv/hops/hadoop-3.2.0.12-EE-SNAPSHOT/share/hadoop/hdfs/lib/log4j-slf4j-impl-2.19.0.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.apache.logging.slf4j.Log4jLoggerFactory]


## <span style="color:#ff5f27">🗄️ Model Loading </span>

In [5]:
# Load the Sentence Transformer
sentence_transformer = SentenceTransformer(
    'all-MiniLM-L6-v2',
).to('cuda')

.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [6]:
# Load a model from the saved model directory
model = AutoPeftModelForCausalLM.from_pretrained(
  saved_model_dir,
  device_map="auto",
  torch_dtype=torch.float16,
)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
# Load the tokenizer from the saved model directory
tokenizer = AutoTokenizer.from_pretrained(
    saved_model_dir,
)

# Set the pad token to the end-of-sequence token
tokenizer.pad_token = tokenizer.eos_token

# Set the padding side to "right" to remove warnings
tokenizer.padding_side = "right"

## <span style="color:#ff5f27">⛓️ LLM Chain </span>

In [8]:
# Create a text generation pipeline using the loaded model and tokenizer
text_generation_pipeline = transformers.pipeline(
    model=model,                          # The pre-trained language model for text generation
    tokenizer=tokenizer,                  # The tokenizer corresponding to the language model
    task="text-generation",               # Specify the task as text generation
    temperature=0.2,                      # Controls the randomness of the generation (higher values for more randomness)
    repetition_penalty=1.5,               # Controls the penalty for repeating tokens in generated text
    return_full_text=True,                # Return the full generated text instead of just the generated tokens
    max_new_tokens=750,                   # Limit the maximum number of newly generated tokens
    pad_token_id=tokenizer.eos_token_id,  # Use the end-of-sequence token as the padding token
    do_sample=True,                       # Enable sampling during text generation
)

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [9]:
# Define a template for generating prompts
prompt_template = """
[INST] 
Instruction: Prioritize brevity and clarity in responses. 
Avoid unnecessary repetition and keep answers concise, adhering to a maximum of 750 characters. 
Eliminate redundant phrases and sentences. 
If details are repeated, provide them only once for better readability. 
Focus on delivering key information without unnecessary repetition. 
If a concept is already conveyed, there's no need to restate it. Ensure responses remain clear and to the point.
Make sure you do not repeat any sentences in your answer.
[/INST]

Previous conversation:
{chat_history}

### CONTEXT:

{context}

### QUESTION:
[INST]{question}[/INST]"""

In [10]:
# Create a Hugging Face pipeline for Mistral LLM using the text generation pipeline
mistral_llm = HuggingFacePipeline(
    pipeline=text_generation_pipeline,
)

In [11]:
# Create prompt from prompt template 
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=prompt_template,
)

In [12]:
# Create a ConversationBufferWindowMemory with specified configuration
memory = ConversationBufferWindowMemory(
    k=3,                         # Number of turns to remember in the conversation buffer
    memory_key="chat_history",   # Key to store the conversation history in memory
    input_key="question",        # Key to access the input question in the conversation
)

In [13]:
# Create LLM chain 
llm_chain = LLMChain(
    llm=mistral_llm, 
    prompt=prompt,
    verbose=False,
    memory=memory,
)

## <span style="color:#ff5f27;"> 🧬 Reranking </span>

In [14]:
def get_reranker():
    reranker = FlagReranker(
        'BAAI/bge-reranker-large', 
        use_fp16=True,
    ) 
    return reranker

In [15]:
# Retrieve a reranker
reranker = get_reranker()

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/801 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

---

## <span style="color:#ff5f27;">🗄️ Context Retrieval </span>

In [16]:
# User Question Example
user_input = 'What are the best risk reporting practices?' 

In [17]:
# Retrieve reranked context and source
context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

## <span style="color:#ff5f27;">🚀 Model Inference </span>

In [18]:
# Generate model response
model_output = llm_chain.invoke({
    "context": context, 
    "question": user_input,
})

print(model_output['text'].split('### RESPONSE:\n')[-1] + source)

The best risk reporting practices involve generating aggregate and up-to-date risk data quickly and efficiently, covering all significant risk areas and providing customized information tailored to the recipient's needs. Additionally, frequent updates and incorporating forward-looking assessments are essential. Clear communication and comprehensibility are crucial factors in ensuring helpful and productive risk reporting. Coverage must extend beyond basic financial metrics to address important aspects such as asset quality, cash flow, earnings per share, equity valuation, interest rates, exchange rates, commodities prices, legal issues, tax laws, among others. Adapting to evolving demands and supervisor queries is equally vital. Finally, integrating robust systems and controls into risk reporting helps improve accuracy and mitigate fraudulent activities. Proper training programs for staff handling sensitive financial documents and IT infrastructure capabilities are fundamental prerequi

---

In [19]:
user_input = 'What is Adaptability?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
    "context": context, 
    "question": user_input,
})

print(model_output['text'].split('### RESPONSE:\n')[-1] + source)

Adaptability refers to flexibility and ability to aggregate risk data according to requesteds scenarios or economic events. It allows for efficient assessment, quick decisions, and supporting stress testing and scenario analyzes. Also, it caters to changes in organization structure, threats sources, transmission mechanisms, trusted paths, and trustworthiness attributes.

References:
 - bcbs239.pdf(https://drive.google.com/file/d/1lPbO28MoOc7XxaSbvpJiaC6VAeUwTqyA/preview?usp=drivesdk): Page: 17, Paragraph: 1
 - NIST.SP.800-53r5.pdf(https://drive.google.com/file/d/1i_pJ-RfbJKw1McBmkoe8d9DCDi-2C-pS/preview?usp=drivesdk): Page: 449, Paragraph: 2
 - pub-ch-bank-supervision-process.pdf(https://drive.google.com/file/d/160FaGKMZPXDGkR_BUDysf-AtZ1eyuWWW/preview?usp=drivesdk): Page: 76, Paragraph: 4


In [20]:
user_input = 'What is a risk management?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
    "context": context, 
    "question": user_input,
})

print(model_output['text'].split('### RESPONSE:\n')[-1] + source)

Risk management involves identifying, measuring, and controlling various forms of risk faced by a bank. It encompasses processes related to Credit, Interest Rate, Liquidity, Price, Operational, Compliance, Strategic, and Reputation risks.

References:
 - pub-ch-bank-supervision-process.pdf(https://drive.google.com/file/d/160FaGKMZPXDGkR_BUDysf-AtZ1eyuWWW/preview?usp=drivesdk): Page: 128, Paragraph: 2
 - pub-ch-bank-supervision-process.pdf(https://drive.google.com/file/d/160FaGKMZPXDGkR_BUDysf-AtZ1eyuWWW/preview?usp=drivesdk): Page: 129, Paragraph: 2
 - pub-ch-community-bank-supervision.pdf(https://drive.google.com/file/d/1ZKd_V5cl5RvPa_DJiKxXKPvCuueCt6Fg/preview?usp=drivesdk): Page: 18, Paragraph: 3


In [21]:
user_input = 'What is the purpose of maintaining an up-to-date data-flow diagram?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
    "context": context, 
    "question": user_input,
})

print(model_output['text'].split('### RESPONSE:\n')[-1] + source)

The purpose of maintaining an up-to-date data-flow diagram is to prevent account data from being overlooked and knowingly left unsecure. It also helps identify critical connection points and control flows within the network.

References:
 - PCI-DSS-v4_0.pdf(https://drive.google.com/file/d/1ULFjxgNXNVFmZnDc3r69iKtUUvYmUIAu/preview?usp=drivesdk): Page: 50, Paragraph: 2
 - PCI-DSS-v4_0.pdf(https://drive.google.com/file/d/1ULFjxgNXNVFmZnDc3r69iKtUUvYmUIAu/preview?usp=drivesdk): Page: 317, Paragraph: 2
 - PCI-DSS-v4_0.pdf(https://drive.google.com/file/d/1ULFjxgNXNVFmZnDc3r69iKtUUvYmUIAu/preview?usp=drivesdk): Page: 277, Paragraph: 2


In [22]:
user_input = 'Why are security and privacy controls important?'

context, source = get_context_and_source(
    user_input, 
    sentence_transformer,
    feature_view, 
    reranker,
)

model_output = llm_chain.invoke({
    "context": context, 
    "question": user_input,
})

print(model_output['text'].split('### RESPONSE:\n')[-1] + source)

Security and privacy controls play a crucial role in protecting organizational operations, assets, individuals, other organizations, and the nation against potential adversaries. They ensure compliance with mandates such as Office of Management and Budget circulars and legislations like the Federal Information Security Modernization Act. By using appropriate security and privacy controls, organizations aim to achieve trustworthy, secure, and resilient systems.

References:
 - NIST.SP.800-53r5.pdf(https://drive.google.com/file/d/1i_pJ-RfbJKw1McBmkoe8d9DCDi-2C-pS/preview?usp=drivesdk): Page: 29, Paragraph: 1
 - NIST.SP.800-53r5.pdf(https://drive.google.com/file/d/1i_pJ-RfbJKw1McBmkoe8d9DCDi-2C-pS/preview?usp=drivesdk): Page: 34, Paragraph: 1
 - NIST.SP.800-53r5.pdf(https://drive.google.com/file/d/1i_pJ-RfbJKw1McBmkoe8d9DCDi-2C-pS/preview?usp=drivesdk): Page: 37, Paragraph: 1


---